## Conjunto de datos

Para este avance se utilizara el dataset de "Sentiment Analysis for Mental Health" el cual esta compuesto por 53042 ejemplos con 7 clasificaciones diferentes

https://www.kaggle.com/datasets/suchintikasarkar/sentiment-analysis-for-mental-health/data


In [1]:
import util_reducido as util
import time
import os
import numpy as np

from copy import deepcopy
import pandas as pd
import psutil
start_time = time.time()

# obtencion de recursos
process = psutil.Process(os.getpid())
start_mem = process.memory_info().rss / 1024 / 1024  # En MB
start_cpu = psutil.cpu_percent(interval=1)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rhuertap/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rhuertap/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rhuertap/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:

SEED = 42
np.random.seed(SEED) 

# Cargar del dataset
archivo = "Combined_Data.csv"
datos = pd.read_csv(archivo)
col_texto = 'statement'
col_clasificacion='status'

label_topics = datos[col_clasificacion].unique()
n_topics = len(label_topics)

# Se eliminan filas donde 'texto' o 'clasificacion' estén vacíos o nulos
datos = datos.dropna(subset=[col_texto, col_clasificacion])  # Se eliminan filas con valores NaN
datos = datos[datos[col_texto].str.strip() != '']   # Se eliminan filas con texto vacío
datos = datos[datos[col_clasificacion].str.strip() != '']      # Se eliminan filas sin clasificación válida

datos[col_texto] = datos[col_texto].apply(lambda x: util.preprocess_text(x))


start_time = time.time()  

all_models = []


X_ent, X_val, X_prueba, y_ent, y_val, y_prueba  = util.separa_datos (datos, col_texto, col_clasificacion)
# Transformar etiquetas
y_train_enc, y_val_enc,  y_test_enc, label_encoder = util.transforma_etiquetas (y_ent, y_val, y_prueba)


print(f"\nVectorizaciones ")
X_train_tfidf, X_val_tfidf, X_test_tfidf, vectorizer_tfidf = util.process_tfidf(X_ent, X_val, X_prueba)
X_train_lda, X_val_lda, X_test_lda, vectorizer_lda, lda_model = util.process_lda(X_ent, X_val, X_prueba, n_topics)
X_train_vader, X_val_vader, X_test_vader, vader_vectorizer = util.process_vader(X_ent, X_val, X_prueba)


#Modelos a entrenar

print(f"Modelos: ")
# Random Forest
all_models = util.train_random_forest_fijo(X_train_lda, X_val_lda , X_train_vader, X_val_vader, X_train_tfidf, X_val_tfidf, y_train_enc,  y_val_enc  ,all_models)
# Logistic Regression
all_models = util.train_logreg_models_fijo (X_train_lda, X_val_lda , X_train_vader, X_val_vader, X_train_tfidf, X_val_tfidf, y_train_enc,  y_val_enc ,all_models)
# Knn
all_models = util.train_knn_models_fijo (X_train_lda, X_val_lda , X_train_vader, X_val_vader, X_train_tfidf, X_val_tfidf, y_train_enc,  y_val_enc  ,all_models)
# Mlp
all_models = util.train_mlp_models_fijo (X_train_lda, X_val_lda , X_train_vader, X_val_vader, X_train_tfidf, X_val_tfidf, y_train_enc,  y_val_enc ,all_models)
# XGB
all_models = util.train_xgb_models_fijo (X_train_lda, X_val_lda , X_train_vader, X_val_vader, X_train_tfidf, X_val_tfidf, y_train_enc,  y_val_enc  ,all_models)



# se juntan modelos
X_prueba_dict = {
    "lda": X_test_lda,
    "vader": X_test_vader,
    "tfidf": X_test_tfidf
}

# Junta todos los modelos entrenados y los ordena
all_models_sorted = sorted(all_models, key=lambda x: x[1], reverse=True)
for modelo in all_models_sorted:
    model_name = modelo[3]
    caracteristica = modelo[2]
    f1_score = modelo[1]
    print(f"{model_name} - {caracteristica} - f1-score: {f1_score}")


historial, fan, mejores_modelos = util.ablacion_iterativa_completa(
    modelos_ordenados=all_models_sorted[:13],
    X_prueba_dict=X_prueba_dict,
    y_prueba=y_test_enc,
    entrenar_y_evaluar_fn=util.entrenar_y_evaluar_simple,
    min_modelos=4,
    label_encoder=label_encoder)


f1_macro = historial["Macro_F1_Score"].iloc[0]
print(f"Macro F1-score: {f1_macro:.4f}")

end_time = time.time()
tiempo_total = end_time - start_time

# Uso de memoria
memoria_mb = psutil.Process().memory_info().rss / (1024 ** 2)

# Guardar métricas en lista
metricas_fan = []
metricas_fan.append({
    "f1_macro": f1_macro,
    "tiempo_m": tiempo_total/60,
    "memoria_mb": memoria_mb
})



df_metricas = pd.DataFrame(metricas_fan)
df_metricas.to_csv("metricas_FAN.csv", index=False)
print("\nArchivo 'metricas_FAN.csv' guardado con éxito.")




Vectorizaciones 
Modelos: 
finaliza el entrenamiento con lda
finaliza el entrenamiento con vader
finaliza el entrenamiento con tfidf
finaliza el entrenamiento con lda
finaliza el entrenamiento con vader
finaliza el entrenamiento con tfidf
finaliza el entrenamiento con lda  + knn
finaliza el entrenamiento con vader  + knn
finaliza el entrenamiento con tfidf + knn
finaliza el entrenamiento con lda  + mlp
finaliza el entrenamiento con vader  + mlp
finaliza el entrenamiento con tfidf + mlp
finaliza el entrenamiento con lda + xgb
finaliza el entrenamiento con vader + xgb
finaliza el entrenamiento con tfidf + xgb
xgb - tfidf - f1-score: 0.658721711474121
mlp - tfidf - f1-score: 0.6478589863435484
logreg - tfidf - f1-score: 0.6358408098714543
random_forest - lda - f1-score: 0.5158822839125463
xgb - lda - f1-score: 0.49238380661777853
knn - lda - f1-score: 0.4864926167254291
random_forest - tfidf - f1-score: 0.46223579764665185
mlp - lda - f1-score: 0.452254381081731
knn - vader - f1-score: 0

# Aplicacion de LIME

In [4]:
for modelo, f1_score, caracteristica, model_name in mejores_modelos:
    print(model_name, caracteristica)

xgb tfidf
mlp tfidf
logreg tfidf
random_forest lda
xgb lda
knn lda
random_forest tfidf
mlp lda
knn vader
random_forest vader
xgb vader
mlp vader


### Obtencion de textos candidatos a usar con LIME

In [5]:
resultados = []

# Creamos un diccionario para las representaciones de prueba
X_prueba_dict = {
    "lda": X_test_lda,
    "vader": X_test_vader,
    "tfidf": X_test_tfidf
}

X_prueba_texto = X_prueba  

# iteracióno de los mejores modelos
for modelo, f1_score, caracteristica, model_name in mejores_modelos:
    if caracteristica not in X_prueba_dict:
        continue  

    y_pred = modelo.predict(X_prueba_dict[caracteristica]) # prediccion de acuerdo a la caracteristica correcta
    
    # regresar a las etiquetas originales
    y_pred_labels = label_encoder.inverse_transform(y_pred)

    for i, texto_original in enumerate(X_prueba_texto):
        resultados.append({
            "texto": texto_original,
            "modelo": model_name,
            "caracteristica": caracteristica,
            "prediccion": y_pred_labels[i],
            "real": label_encoder.inverse_transform([y_test_enc[i]])[0]  # Aquí traemos la clase real
        })

df_resultados = pd.DataFrame(resultados)

df_resultados.to_csv("resultados_mejores_modelos.csv", index=False)
print("Predicciones múltiples generadas y guardadas.")

# Agrupamos los resultados por cada texto y su clase real, 
# de manera que tengamos la lista de predicciones de los distintos modelos por texto
agrupado = df_resultados.groupby(['texto', 'real'])['prediccion'].apply(list).reset_index()

# Agregamos una columna con el número de palabras de cada texto (para filtrar textos demasiado cortos o largos)
agrupado['num_palabras'] = agrupado['texto'].apply(lambda x: len(x.split()))

# Nos quedamos solo con los textos "interesantes" (ni muy cortos ni muy largos)
agrupado = agrupado[(agrupado['num_palabras'] >= 10) & (agrupado['num_palabras'] <= 100)]

# Seleccionamos una muestra aleatoria de textos con consenso y 10 con discrepancia en sus clasificaciones 
agrupado['consenso'] = agrupado['prediccion'].apply(lambda x: len(set(x)) == 1)
casos_consenso = agrupado[agrupado['consenso'] == True].sample(10, random_state=SEED)
casos_discrepancia = agrupado[agrupado['consenso'] == False].sample(10, random_state=SEED)

# Unimos ambos subconjuntos para generar un set de candidatos para explicación con LIME
casos_candidatos = pd.concat([casos_consenso, casos_discrepancia])

# Guardamos el archivo con los textos seleccionados para análisis posteriores
casos_candidatos.to_csv("candidatos_LIME.csv", index=False)
print("Candidatos para LIME generados y guardados.")

Predicciones múltiples generadas y guardadas.
Candidatos para LIME generados y guardados.


## Implementacion de LIME 

In [ ]:
from lime.lime_text import LimeTextExplainer
import numpy as np

# Uso de especializado de LIME para generar las perturbaciones a nivel de palabras
explainer = LimeTextExplainer(class_names=label_encoder.classes_)

vectorizers = {
    "tfidf": vectorizer_tfidf,  
    "vader": vader_vectorizer,
    "lda": (vectorizer_lda, lda_model) 
}

# se toman las oraciones candidatas 
for idx, fila in casos_candidatos.iterrows():
    texto = fila['texto']
    clase_real = fila['real']  # Se obtiene la clase real del texto
    print(f"\n____________________\nTexto {idx+1}: {texto}\n")
    
    for modelo, f1_score, caracteristica, model_name in mejores_modelos:
        
        if caracteristica not in vectorizers:
            continue

        print(f"Modelo: {model_name} ({caracteristica})")

        # se construye el predictor que LIME usara para evaluar el modelo sobre cada variante del texto
        #     X_text: vectorización del texto original en la representación correspondiente (TF-IDF, VADER o LDA)
        #     predictor: función adaptadora que usara LIME  para evaluar variantes del texto
        X_text, predictor = util.contruye_predictor_LIME (caracteristica, texto, modelo, vectorizers, label_encoder)
        
        # Ejecutamos LIME: el cambio crítico -> pedimos explicación de todas las clases
        exp = explainer.explain_instance(
            texto,                           # texto: parrafo que queremos explicar y entender
            predictor,                       
            num_features=10,       # Indica cuántas palabras destacará LIME como las que más influyeron en la predicción para este ejemplo
            top_labels=len(explainer.class_names)  # top_labels: cantidad de clases para las cuales generará explicación, aquí usamos todas
        )
        
        # Obtenemos la clase predicha por el modelo en turno
        pred_clase = modelo.predict(X_text)[0]
        pred_clase_nombre = label_encoder.inverse_transform([pred_clase])[0]

        # Calculamos el índice posicional para LIME
        lime_class_index = list(explainer.class_names).index(pred_clase_nombre)

        # Imprimimos todo para análisis
        print(f"Clase real: {clase_real}")
        print(f"Clase predicha: {pred_clase_nombre}")

        # Mostramos la explicación centrada en la clase predicha
        exp.show_in_notebook(labels=[lime_class_index])

In [1]:
import platform
import psutil

# Información básica del sistema
print("Processor:", platform.processor())
print("Machine:", platform.machine())
print("Platform:", platform.platform())
print("Architecture:", platform.architecture())

# Núcleos de CPU
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores (logical):", psutil.cpu_count(logical=True))

# Frecuencia
cpu_freq = psutil.cpu_freq()
print("Max Frequency:", cpu_freq.max, "MHz")
print("Min Frequency:", cpu_freq.min, "MHz")
print("Current Frequency:", cpu_freq.current, "MHz")

Processor: arm
Machine: arm64
Platform: macOS-15.5-arm64-arm-64bit
Architecture: ('64bit', '')
Physical cores: 8
Total cores (logical): 8
Max Frequency: 3204 MHz
Min Frequency: 600 MHz
Current Frequency: 3204 MHz
